In [1]:
import pandas as pd
import numpy as np
import time
data=pd.read_csv('input.csv',sep=',')
data.head()
machines = ['m1','m2','m3','m4','m5']
#convert into dictionary with first column as key and rest as values
# data_dict = data.set_index('Run').T.to_dict('list')
# jobs=data_dict
# del data, data_dict

In [19]:

# # Example usage - give data here
# jobs = { # Machines: M1, M2, M3
#     'ERR016294': [9, 9, 10], #j1
#     'ERR016314': [6,10,9], #j2
#     'ERR016316': [11,7,12], #j3
# }  # Example job durations
# machines = ['m1', 'm2', 'm3']  # Example machines

##  Gredy Strategy for Plan Generation

In [22]:
import pandas as pd

# Read the data
data = pd.read_csv('input.csv', sep=',')
jobs = data.set_index('Run').T.to_dict('list')
machines = ['M1', 'M2', 'M3', 'M4', 'M5']

def greedy_schedule(jobs, machines):
    execution_plan = {machine: [('BEGIN', 0)] for machine in machines}
    execution_time = {machine: 0 for machine in machines}
    execution_order = []
    J = list(jobs.keys())
    while J:
        T = []
        for job in J:
            for i, time in enumerate(jobs[job]):
                T.append((job, machines[i], time))
        J_alpha, m_beta, w = min(T, key=lambda x: (execution_time[x[1]], x[2]))
        execution_plan[m_beta].append((f'EXEC {J_alpha}', w))
        execution_time[m_beta] += w
        execution_order.append((J_alpha, m_beta, w))
        J.remove(J_alpha)
    for machine in machines:
        execution_plan[machine].append(('END', 0))
        # print(execution_plan[machine])
    return execution_plan, execution_time, execution_order

execution_plan, execution_time, execution_order = greedy_schedule(jobs, machines)


In [23]:
# Print the results
for machine, plan in execution_plan.items():
    print(f"{machine}: {', '.join([f'{task} ({time} units)' for task, time in plan])}")
print("\nExecution times:")
for machine, time in execution_time.items():
    print(f"{machine}: {time} units")
print("\nExecution order:")
for job, machine, time in execution_order:
    print(f"{job} executed on {machine} for {time} units")
    
# Write the results to files
for machine, plan in execution_plan.items():
    with open(f'script_{machine}.txt', 'w') as file:
        for task, _ in plan:
            file.write(f"{task}\n")

print("Execution plans have been written to files.")

M1: BEGIN (0 units), EXEC ERR062966 (716 units), EXEC ERR016338 (935 units), EXEC ERR016327 (1235 units), EXEC ERR020276 (1244 units), END (0 units)
M2: BEGIN (0 units), EXEC ERR062953 (1059 units), EXEC SRR794387 (1516 units), EXEC ERR020241 (4521 units), END (0 units)
M3: BEGIN (0 units), EXEC SRR799760 (420 units), EXEC ERR016326 (601 units), EXEC ERR019897 (757 units), EXEC ERR018416 (2193 units), END (0 units)
M4: BEGIN (0 units), EXEC ERR068392 (579 units), EXEC ERR019492 (667 units), EXEC ERR020289 (763 units), EXEC ERR020264 (2457 units), END (0 units)
M5: BEGIN (0 units), EXEC ERR062940 (812 units), EXEC ERR018544 (858 units), EXEC ERR022463 (1629 units), END (0 units)

Execution times:
M1: 4130 units
M2: 7096 units
M3: 3971 units
M4: 4466 units
M5: 3299 units

Execution order:
SRR799760 executed on M3 for 420 units
ERR068392 executed on M4 for 579 units
ERR062966 executed on M1 for 716 units
ERR062940 executed on M5 for 812 units
ERR062953 executed on M2 for 1059 units
ERR016

## Calculating Actual for Predicted

In [ ]:

# # create a dataframe with duration, machine and job name for above
# df = pd.DataFrame([(job, machine, duration) for machine, durations in execution_plan.items() for job, duration in zip(job_allocation[machine], durations)], columns=['Job', 'Machine', 'Predicted Duration'])
# # print(df)actual=pd.read_csv('greedy_actual_S1.csv',sep='\t');actual.set_index("Unnamed: 0", inplace=True)
# output=df
# import warnings
# warnings.filterwarnings("ignore")
# output["Machine"].replace("m1", "1T4GPU (M0)",inplace=True)
# output["Machine"].replace("m2", "2T4GPU (M1)",inplace=True)
# output["Machine"].replace("m3", "1RTX GPU (M2)",inplace=True)
# output["Machine"].replace("m4", "2RTX GPU (M3)",inplace=True)
# output["Machine"].replace("m5", "3RTX GPU (M4)",inplace=True)
# # add a column in output 
# output["Actual Duration"] = 0
# for i in range(len(output)):
#     output["Actual Duration"][i] = actual.loc[output["Job"][i], output["Machine"][i]]
# output.columns = ["Job", "Predicted Machine", "Predicted Duration", "Actual Duration"]
# output
## group by Machine and sum Duration and Actual Duration
# grouped = df.groupby('Predicted Machine').agg({
#     'Predicted Duration': 'sum',
#     'Actual Duration': 'sum'
# }).reset_index()

# # Rename columns for clarity
# grouped.columns = ['Machine', 'Predicted (Sum)', 'Actual Duration (Sum)']

# # Display the result
# print(grouped)

## Plan Execution


In [ ]:
# Input ei - Execution plan for machine mi
# Output: Gives execution schedule for VMs and Executes the schedule on machines
import subprocess
execution_plan, job_allocation = greedy_schedule(jobs, machines)
start= time.time() 
# pushing job on machine 
vms = ['vm1', 'vm2', 'vm3', 'vm4', 'vm5']
for i in range(len(machines)):
    # subprocess.run(["ssh", vms[i+1], "touch", "script.txt"])
    # append this script.txt file with job_allocation[machines[i]]
    ei = job_allocation[machines[i]]
    with open("script_{}.txt".format(machines[i]), "w") as f:
        f.write("BEGIN\n")
        for j in ei:
            f.write("EXEC " + j + "\n")
        f.write("END\n")
    # ssh vm1 script.txt >> ei
print("Copied scheduling plan on all machines")
print("Starting job execution on all machines")
for i in range(len(machines)):
    # ssh vm1 python3 script.py
    print(f"Jobs on Machine {machines[i]}: {job_allocation[machines[i]]}")
end = time.time()
print("Time taken for greedy schedlule workload is: ", round(end-start,3))